In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import seaborn as sns


In [ ]:
df_log=pd.read_pickle('src/data/testA/totalExposureLog.pkl')
# df=pd.read_pickle('src/data/testA/user_data.pkl')
df_op=pd.read_pickle('src/data/testA/ad_operation.pkl')
df_ad=pd.read_pickle('src/data/testA/ad_static_feature.pkl')
df_test=pd.read_pickle('src/data/testA/test_sample.pkl')

In [ ]:
df_log_ad = df_log[['aid', 'request_timestamp']].drop_duplicates()

In [ ]:
df_log_ad['date'] = pd.to_datetime(df_log_ad['request_timestamp'], unit='s').dt.date
df_log_ad = df_log_ad[['aid', 'date']].drop_duplicates()
df_log_ad['date'].drop_duplicates().nunique()

In [ ]:
df_log_ad_min = df_log_ad.groupby(['aid'])[['date']].min().reset_index()
df_log_ad= pd.merge(df_log_ad,df_log_ad_min,on='aid') 
df_log_ad['date_min'] = df_log_ad['date_y']
df_log_ad['date'] = df_log_ad['date_x']
df_log_ad['isnew'] = df_log_ad['date'] == df_log_ad['date_min']
del df_log_ad['date_x']
del df_log_ad['date_y']
df_log_ad

In [ ]:
f, ax = plt.subplots(figsize=(15, 5))
ax = sns.countplot(data=df_log_ad, x='date')
f.autofmt_xdate()


In [ ]:
f, ax = plt.subplots(figsize=(15, 5))
ax = sns.histplot(data=df_log_ad, x="date",hue="isnew", multiple="stack")
f.autofmt_xdate()

In [ ]:
df_log_ad.nunique()

In [ ]:
# 广告静态数据
df_ad['create_date'] = pd.to_datetime(df_ad['create_timestamp'], unit='s').dt.date
df_ad.nunique()

In [ ]:
df_ad_log = pd.merge(df_ad,df_log_ad,on='aid') 
df_ad_log.nunique()

In [ ]:
# 广告操作数据
df_op=pd.read_pickle('src/data/testA/ad_operation.pkl')
df_op_ad = pd.merge(df_op,df_ad_log,on='aid') 

In [ ]:
df_op_ad.nunique()

In [ ]:
df_op_ad['create_time'] = pd.to_datetime(df_op_ad['create_timestamp'], unit='s')
df_op_ad['motify_time'] = df_op_ad.apply(lambda x: x['request_time'] if x['type'] == 1 else x['create_time'], axis=1)

In [ ]:
df_op_ex = df_op_ad[['aid','create_time','motify_time','type','op_type','value']].drop_duplicates().reset_index()[['aid','create_time','motify_time','type','op_type','value']]

In [ ]:
df_op_ex.to_pickle('src/data/testA/ad_operation_filter_extend.pkl')

In [ ]:
df_op

In [ ]:
#更改广告状态
df_op_status = df_op_ex[df_op_ex['op_type'] == 1]
df_op_status.to_pickle('src/data/testA/ad_operation_filter_status.pkl')

In [ ]:
df_op_status['motify_date'] = df_op_status['motify_time'].dt.date
f, ax = plt.subplots(figsize=(15, 5))
ax = sns.histplot(data=df_op_status[df_op_status['motify_date'] > pd.to_datetime('1970-01-01')], x="motify_date",hue="value", multiple="stack")
f.autofmt_xdate()

In [ ]:
df_op_status['time'] = df_op_status['motify_time'].dt.hour
f, ax = plt.subplots(figsize=(15, 5))
ax = sns.histplot(data=df_op_status[df_op_status['motify_time'] > pd.to_datetime('2019-02-16')], x="time",hue="value", multiple="stack")

In [ ]:
#更改出价
df_op_bid = df_op_ex[df_op_ex['op_type'] == 2]
df_op_bid.to_pickle('src/data/testA/ad_operation_filter_bid.pkl')
df_op_bid = pd.read_pickle('src/data/testA/ad_operation_filter_bid.pkl')

In [ ]:
df_op_bid['bid'] = pd.to_numeric(df_op_bid['value'])

In [ ]:
df_op_bid['motify_date'] = df_op_bid['motify_time'].dt.date
f, ax = plt.subplots(figsize=(15, 5))
ax = sns.histplot(data=df_op_bid[df_op_bid['motify_date'] > pd.to_datetime('1970-01-01')],x='bid',log_scale=True )


In [ ]:
df_op_bid

In [ ]:
df_op_bid['motify_date'] = df_op_bid['motify_time'].dt.date
df_op_bid_nor = df_op_bid[df_op_bid['motify_date'] > pd.to_datetime('2019-03-10')]
bid_counts = df_op_bid_nor.groupby(['motify_date','aid'])['value'].nunique().reset_index()
f, ax = plt.subplots(figsize=(15, 5))
ax = sns.violinplot(data=bid_counts,x='motify_date',y='value')
f.autofmt_xdate()

In [ ]:
bid_counts

In [ ]:
df_op_bid_nor

In [ ]:
#更改投放时段
df_op_periods = df_op_ex[df_op_ex['op_type'] == 4]
df_op_periods.to_pickle('src/data/testA/ad_operation_filter_periods.pkl')

In [ ]:
df_op_periods

In [ ]:
#更改人群定向
df_op_crowd = df_op_ex[df_op_ex['op_type'] == 3]
df_op_crowd.to_pickle('src/data/testA/ad_operation_filter_crowd.pkl')

In [ ]:
df_test.info()

In [ ]:
df_test_op_bid = pd.merge(df_test,df_op_bid,on=['aid'],how='left') 
df_test_op_bid

In [ ]:
df_test_op_bid.isnull().any(axis=1).sum()